In [13]:
!pip install -r requirements.txt
#installing depedencies

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 10.3 MB/s eta 0:00:00


In [27]:
#importing libraries
import numpy as np
import pandas as pd 
import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.losses import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from tensorflow import keras

In [15]:
#load the dataset
lcp_train = pd.read_csv("lcp_single_train.tsv.txt", sep="\t")
lcp_test=pd.read_csv("lcp_single_test.tsv.txt", sep="\t")

#Here im reloading the train dataset just to get construct the tokenizer, as I want the test test to have the same dictionary and padding as train dataset.

In [16]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(lcp_train['sentence'])
#loading tokenizer and fitting on train dataset

In [17]:
#converting test sentences to sequence
sent_seq = tokenizer.texts_to_sequences(lcp_train['sentence'])
max_seq_len = max(len(seq) for seq in sent_seq)


In [18]:
#converting test sentences to sequences
sent_seq_test = tokenizer.texts_to_sequences(lcp_test['sentence'])
sent_seq_test = pad_sequences(sent_seq_test, maxlen=max_seq_len)

In [19]:
#here converting the train words to sequence, just to get the max_length, which is used in the model training

lcp_train.token=lcp_train.token.astype(str)
token = tokenizer.texts_to_sequences(lcp_train['token'])
max_word_len = max(len(w) for w in token)

In [20]:
#converting test tokens into sequences
lcp_test.token=lcp_test.token.astype(str)
token_test = tokenizer.texts_to_sequences(lcp_test['token'])
token_test = pad_sequences(token_test, maxlen=max_word_len)

In [21]:
#converting it to one hot encoding
onehot_encoder = OneHotEncoder(sparse=False)
corpus_encoded_test = onehot_encoder.fit_transform(lcp_test['corpus'].values.reshape(-1, 1))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [28]:
#ref : https://www.tensorflow.org/guide/keras/save_and_serialize
load_base_model = keras.models.load_model("Stack_RNN.h5")

In [29]:
test_y=lcp_test['complexity'].values
# Evaluate the model
test_pred = load_base_model.predict([sent_seq_test, token_test, corpus_encoded_test])
#print(test_pred,test_y)
mae = np.mean(np.abs(test_pred - test_y))
print("Mean Absolute Error(Without tuning):", mae)

28/28 [==============================] - 7s 204ms/step
Mean Absolute Error(Without tuning): 0.11000688717584296


In [30]:
load_tuned_model = keras.models.load_model("Tuned_Stack_RNN.h5")
# Evaluate the model
test_pred = load_tuned_model.predict([sent_seq_test, token_test, corpus_encoded_test])
#print(test_pred,test_y)
mae = np.mean(np.abs(test_pred - test_y))
print("Mean Absolute Error(With tuning):", mae)

28/28 [==============================] - 11s 315ms/step
Mean Absolute Error(With tuning): 0.1072609143607685
